In [8]:
import supervision as sv
from ultralytics import YOLO
from tqdm import tqdm
import numpy as np

In [9]:
def process_video(
	source_weights_path: str, 
	source_video_path: str,
	target_video_path: str, 
	confidence_threshold: float = 0.3,
	iou_threshold: float = 0.7
) -> None:
	model = YOLO(source_weights_path)       # Load YOLO model 
	classes = list(model.names.values())    # Class names 
	tracker = sv.ByteTrack()                # Bytetracker instance 
	box_annotator = sv.BoxAnnotator()     # BondingBox annotator instance 
	label_annotator = sv.LabelAnnotator()         # Label annotator instance 
	frame_generator = sv.get_video_frames_generator(source_path=source_video_path) # for generating frames from video
	video_info = sv.VideoInfo.from_video_path(video_path=source_video_path)

	with sv.VideoSink(target_path=target_video_path, video_info=video_info) as sink:
		for frame in tqdm(frame_generator, total= video_info.total_frames):
			# Getting result from model
			results = model(frame, verbose=False, conf= confidence_threshold, iou = iou_threshold)[0] 
			detections = sv.Detections.from_ultralytics(results)    # Getting detections
			#Filtering classes for car and truck only instead of all COCO classes.
			detections = detections[np.where((detections.class_id==2)|(detections.class_id==7))]
			detections = tracker.update_with_detections(detections)  # Updating detection to Bytetracker
			# Annotating detection boxes
			annotated_frame = box_annotator.annotate(scene = frame.copy(), detections= detections)

			#Prepare labels
			labels = []
			for index in range(len(detections.class_id)):
				# creating labels as per required.
				labels.append("#" + str(detections.tracker_id[index]) + " " + classes[detections.class_id[index]] + " "+ str(round(detections.confidence[index],2)) )
			
			# Annotating labels
			annotated_label_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
			sink.write_frame(frame = annotated_label_frame)

In [10]:
process_video(
	source_weights_path= 'yolov8m.pt',
	source_video_path= 'input.mp4',
	target_video_path= 'test_pred.mp4',
	confidence_threshold=0.1,
	iou_threshold=0.6
)

100%|██████████| 307/307 [00:08<00:00, 35.25it/s]
